In [7]:
import pandas as pd

# Làm sạch dữ liệu

In [8]:
# Đọc file CSV vào dataframe
df = pd.read_csv('data0205.csv')

# Xóa các dòng trùng nhau dựa trên cột "link"
df.drop_duplicates(subset=['link'], inplace=True)

# Lưu lại vào file CSV
df.to_csv('data_done.csv', index=False)

In [9]:
df = pd.read_csv("data_done.csv")

# Tạo đặc trưng mới

In [10]:
df["herald_difference"] = df["blue_heralds"] - df["red_heralds"]
df["dragon_difference"] = df["dragon_blue"] - df["dragon_red"]
df["turret_difference"] = df["turret_blue"] - df["turret_red"]

df["kda_difference_top"] = df["kda_top_blue"] - df["kda_top_red"]
df["kda_difference_jungle"] = df["kda_jungle_blue"] - df["kda_jungle_red"]
df["kda_difference_mid"] = df["kda_mid_blue"] - df["kda_mid_red"]
df["kda_difference_ad"] = df["kda_ad_blue"] - df["kda_ad_red"]
df["kda_difference_sp"] = df["kda_sp_blue"] - df["kda_sp_red"]

# Xóa các cột không cần thiết

In [11]:
df = df.drop(['kda_top_blue', 'kda_top_red', 'kda_jungle_blue', 'kda_jungle_red', 'kda_mid_blue', 'kda_mid_red', 'kda_ad_blue', 'kda_ad_red', 'kda_sp_blue', 'kda_sp_red',
              'link','dragon_blue', 'dragon_red', 'turret_blue', 'turret_red', 'blue_heralds', 'red_heralds','top_blue','top_red', 'mid_blue', 'mid_red','jung_blue','jung_red','ad_blue','ad_red','support_blue','support_red'], axis=1)
df['team_win'] = df['team_win'].replace({'blue': 1, 'red': 0})
df.to_csv("processed_data.csv", index=False)


In [12]:
data = pd.read_csv("processed_data.csv")

In [13]:
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, StandardScaler, RobustScaler, Normalizer, QuantileTransformer, PowerTransformer
from sklearn.ensemble import RandomForestClassifier


In [15]:
# # Tính ma trận tương quan
# corr = data.corr()

# # Vẽ heatmap
# sns.heatmap(corr, cmap='coolwarm', annot=True)
# plt.show()



NameError: name 'sns' is not defined

In [ ]:
# Chia dữ liệu thành features và label
X = data.drop(['team_win'], axis=1)
y = data['team_win']

# Khởi tạo mô hình Random Forest
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100, random_state=42)

# Đào tạo mô hình và tính toán độ quan trọng của các đặc trưng
rfc.fit(X, y)
importances = rfc.feature_importances_

# Trực quan hóa độ quan trọng của các đặc trưng
import matplotlib.pyplot as plt
import seaborn as sns

feature_importances = pd.DataFrame({'feature': X.columns,
                                    'importance': importances})
sns.barplot(x='importance', y='feature', data=feature_importances)
plt.title('Độ quan trọng của các đặc trưng')
plt.xlabel('Độ quan trọng')
plt.ylabel('Đặc trưng')
plt.show()

NameError: name 'data' is not defined

In [ ]:
from sklearn.feature_selection import RFE
from xgboost import XGBClassifier

# Tách dữ liệu thành tập X và tập y
X = data.drop('team_win', axis=1)
y = data['team_win']

# Chia dữ liệu thành tập train và tập test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Khởi tạo mô hình XGBoost
xgb = XGBClassifier()

# Sử dụng RFE để chọn ra các đặc trưng quan trọng
rfe = RFE(xgb)
rfe.fit(X_train, y_train)

# Xếp hạng độ quan trọng của các đặc trưng
ranking = rfe.ranking_

# In ra tên đặc trưng cùng với xếp hạng của nó
for i in range(len(X_train.columns)):
    print(f"Đặc trưng {X_train.columns[i]} - Xếp hạng: {ranking[i]}")

Đặc trưng gold_difference - Xếp hạng: 1
Đặc trưng herald_difference - Xếp hạng: 1
Đặc trưng dragon_difference - Xếp hạng: 1
Đặc trưng turret_difference - Xếp hạng: 5
Đặc trưng kda_difference_top - Xếp hạng: 3
Đặc trưng kda_difference_jungle - Xếp hạng: 1
Đặc trưng kda_difference_mid - Xếp hạng: 6
Đặc trưng kda_difference_ad - Xếp hạng: 2
Đặc trưng kda_difference_sp - Xếp hạng: 4


In [ ]:
from xgboost import XGBClassifier

# Tách dữ liệu thành tập X và tập y
X = data.drop('team_win', axis=1)
y = data['team_win']

# Chia dữ liệu thành tập train và tập test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Các kĩ thuật chuẩn hóa
scalers = [MinMaxScaler(), MaxAbsScaler(), StandardScaler(), RobustScaler(), Normalizer(), QuantileTransformer(n_quantiles=623), PowerTransformer()]

for scaler in scalers:
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Khởi tạo mô hình XGBoost với các tham số cơ bản
    xgb = XGBClassifier(learning_rate=0.1, n_estimators=100, max_depth=5, min_child_weight=1, subsample=0.8, colsample_bytree=0.8, objective='binary:logistic', nthread=4, scale_pos_weight=1, seed=27)

    # Huấn luyện mô hình trên tập train
    xgb.fit(X_train, y_train)

    # Dự đoán kết quả trên tập test
    y_pred = xgb.predict(X_test)

    # Tính toán độ chính xác của mô hình trên tập test
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy:.4f}")

    # Tính toán độ quan trọng của các đặc trưng
    importance = xgb.feature_importances_
    for i in range(len(importance)):
        print(f"Đặc trưng {X.columns[i]}: {importance[i]:.4f}")

Accuracy: 0.7509
Đặc trưng gold_difference: 0.3418
Đặc trưng herald_difference: 0.0753
Đặc trưng dragon_difference: 0.0940
Đặc trưng turret_difference: 0.1436
Đặc trưng kda_difference_top: 0.0547
Đặc trưng kda_difference_jungle: 0.1149
Đặc trưng kda_difference_mid: 0.0538
Đặc trưng kda_difference_ad: 0.0613
Đặc trưng kda_difference_sp: 0.0607
Accuracy: 0.7509
Đặc trưng gold_difference: 0.3418
Đặc trưng herald_difference: 0.0753
Đặc trưng dragon_difference: 0.0940
Đặc trưng turret_difference: 0.1436
Đặc trưng kda_difference_top: 0.0547
Đặc trưng kda_difference_jungle: 0.1149
Đặc trưng kda_difference_mid: 0.0538
Đặc trưng kda_difference_ad: 0.0613
Đặc trưng kda_difference_sp: 0.0607
Accuracy: 0.7520
Đặc trưng gold_difference: 0.3418
Đặc trưng herald_difference: 0.0753
Đặc trưng dragon_difference: 0.0940
Đặc trưng turret_difference: 0.1436
Đặc trưng kda_difference_top: 0.0547
Đặc trưng kda_difference_jungle: 0.1149
Đặc trưng kda_difference_mid: 0.0538
Đặc trưng kda_difference_ad: 0.0613
Đ

In [ ]:
# Lấy các feature cần dùng
X = data.drop(['team_win'], axis=1)

# Lấy label cần dự đoán
y = data['team_win']

# Khởi tạo cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Các kĩ thuật chuẩn hóa
scalers = [MinMaxScaler(), MaxAbsScaler(), StandardScaler(), RobustScaler(), Normalizer(), QuantileTransformer(n_quantiles=623), PowerTransformer()]

# Duyệt qua các kĩ thuật chuẩn hóa
scores = []
mean_feature_importances = np.zeros(X.shape[1])
for scaler in scalers:
    # Chuẩn hóa dữ liệu
    X_scaled = scaler.fit_transform(X)
    
    # Khởi tạo model Random Forest Classifier
    rfc = RandomForestClassifier(n_estimators=100, random_state=42)
    
    # Duyệt qua các fold và tính accuracy trung bình
    acc = []
    feature_importances = np.zeros(X.shape[1])
    for train_index, test_index in kf.split(X_scaled):
        X_train, X_test = X_scaled[train_index], X_scaled[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        rfc.fit(X_train, y_train)
        y_pred = rfc.predict(X_test)
        
        acc.append(accuracy_score(y_test, y_pred))
        feature_importances += rfc.feature_importances_

    mean_feature_importances += feature_importances / kf.n_splits
    scores.append(np.mean(acc))

# Tính trung bình độ quan trọng của các đặc trưng trên tất cả các fold
mean_feature_importances /= len(scalers)

# In ra độ quan trọng của các đặc trưng
for i, importance in enumerate(mean_feature_importances):
    print("Đặc trưng {}: {:.2f}%".format(X.columns[i], importance * 100))

# Tìm kĩ thuật chuẩn hóa tốt nhất
best_score = max(scores)
best_scaler = scalers[scores.index(best_score)]
print(f"Best score: {best_score:.4f}")
print(f"Best scaler: {type(best_scaler).__name__}")


Đặc trưng gold_difference: 30.38%
Đặc trưng herald_difference: 4.79%
Đặc trưng dragon_difference: 5.61%
Đặc trưng turret_difference: 5.97%
Đặc trưng kda_difference_top: 9.32%
Đặc trưng kda_difference_jungle: 12.43%
Đặc trưng kda_difference_mid: 9.87%
Đặc trưng kda_difference_ad: 10.68%
Đặc trưng kda_difference_sp: 10.95%
Best score: 0.7491
Best scaler: StandardScaler


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

# Load data
df = pd.read_csv("processed_data.csv")

# Select features and target
features = ["gold_difference", "herald_difference", "dragon_difference", "turret_difference",
            "kda_difference_top", "kda_difference_jungle", "kda_difference_mid",
            "kda_difference_ad", "kda_difference_sp"]
target = "team_win"

X = df[features]
y = df[target]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define SVM classifier and hyperparameters
svm = SVC(random_state=42)
params = {
    'C': [0.01, 0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf', 'sigmoid']
}

# Tune hyperparameters using grid search
grid_search = GridSearchCV(svm, params, cv=5)
grid_search.fit(X_train_scaled, y_train)

# Print best hyperparameters and best score
print(f"Best hyperparameters: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_:.4f}")

# Evaluate performance on test set
y_pred = grid_search.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy on test set: {accuracy:.4f}")


Best hyperparameters: {'C': 0.1, 'kernel': 'linear'}
Best score: 0.7570
Accuracy on test set: 0.7580


In [ ]:
# Get the features needed
from sklearn.linear_model import LogisticRegression


X = data.drop(['team_win'], axis=1)

# Get the label to be predicted
y = data['team_win']

# Initialize cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)
scores = []

# Standardization techniques
scalers = [MinMaxScaler(), MaxAbsScaler(), StandardScaler(), RobustScaler(), Normalizer(), QuantileTransformer(n_quantiles=623), PowerTransformer()]

# Loop through standardization techniques
for scaler in scalers:
    # Scale the data
    X_scaled = scaler.fit_transform(X)

    # Initialize logistic regression model
    lr = LogisticRegression(random_state=42)

    # Loop through folds and calculate average accuracy
    acc = []
    for train_index, test_index in kf.split(X_scaled):
        X_train, X_test = X_scaled[train_index], X_scaled[test_index]
        y_train, y_test = y[train_index], y[test_index]

        lr.fit(X_train, y_train)
        y_pred = lr.predict(X_test)

        acc.append(accuracy_score(y_test, y_pred))

    scores.append(np.mean(acc))

# Print the average accuracy for each standardization technique
for i in range(len(scalers)):
    print(f"{type(scalers[i]).__name__}: {scores[i]:.4f}")

# Find the best standardization technique
best_score = max(scores)
best_scaler = scalers[scores.index(best_score)]
print(f"Best score: {best_score:.4f}")
print(f"Best scaler: {type(best_scaler).__name__}")

MinMaxScaler: 0.7572
MaxAbsScaler: 0.7577
StandardScaler: 0.7567
RobustScaler: 0.7568
Normalizer: 0.7484
QuantileTransformer: 0.7572
PowerTransformer: 0.7567
Best score: 0.7577
Best scaler: MaxAbsScaler
